In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
from car_detector.hybridnets import HybridNets, optimized_model

In [2]:
VIDEO_FILE = "demo_video/stop.mp4"
model_path = "car_detector/models/hybridnets_384x512/hybridnets_384x512.onnx"
anchor_path = "car_detector/models/hybridnets_384x512/anchors_384x512.npy"

# Initialize road detector
optimized_model(model_path) # Remove unused nodes
car_detector = HybridNets(model_path, anchor_path, conf_thres=0.5, iou_thres=0.5)

/home/chienfeng/.virtualenvs/hackthon/lib/python3.7/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:56: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names))


In [3]:
tracker = IOUTracker(max_lost=2, iou_threshold=0.5, min_detection_confidence=0.4, max_detection_confidence=0.7,
                         tracker_output_format='mot_challenge')

In [4]:
cap = cv.VideoCapture(VIDEO_FILE)
while True:
    ok, image = cap.read()

    if not ok:
        print("Cannot read the video feed.")
        break

    seg_map, filtered_boxes, filtered_scores = car_detector(image)
    class_ids = np.ones_like(filtered_scores).astype(int)
    
    tracks = tracker.update(filtered_boxes, filtered_scores, class_ids=class_ids)

    updated_image = car_detector.draw_2D(image)
    updated_image = draw_tracks(updated_image, tracks)

    cv.imshow("image", updated_image)
    if cv.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv.destroyAllWindows()